[程式碼]('https://www.mygreatlearning.com/blog/pos-tagging/')

內容基本上沒有更改，只有調整一些順序跟備註。

# HMM

In [ ]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time

In [ ]:
nltk.download('treebank')
nltk.download('universal_tagset')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [ ]:
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
print(nltk_data[:2])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')]]


In [ ]:
# for sent in nltk_data[:2]:
#   for tuple in sent:
#     print(tuple)

# train/test

In [ ]:
train_set,test_set =train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state = 101)

In [ ]:
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
print(len(train_tagged_words))
print(len(test_tagged_words))

80310
20366


In [ ]:
train_tagged_words[0]

('Drink', 'NOUN')

# 詞性/單字

In [ ]:
tags = {tag for word,tag in train_tagged_words}     #用set去重複
vocab = {word for word,tag in train_tagged_words}
print(len(tags))
print(tags)

12
{'X', '.', 'ADP', 'PRT', 'ADJ', 'PRON', 'NOUN', 'VERB', 'CONJ', 'NUM', 'DET', 'ADV'}


In [ ]:
# Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
  tag_list = [pair for pair in train_bag if pair[1]==tag] #找出詞性相同
  w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word] #找出相同單字
  count_tag = len(tag_list)
  count_w_given_tag = len(w_given_tag_list)
    
  return (count_w_given_tag, count_tag)

In [ ]:
# Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]  # 所有詞性標記
    count_t1 = len([t for t in tags if t==t1])  # 詞性是t1的單字數量
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:  # 計算t1到t2的數量 
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [ ]:
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(tags_matrix)

[[7.57255405e-02 1.60868734e-01 1.42225638e-01 1.85085520e-01
  1.76821072e-02 5.41995019e-02 6.16951771e-02 2.06419379e-01
  1.03786280e-02 3.07514891e-03 5.68902567e-02 2.57543717e-02]
 [2.56410260e-02 9.23720598e-02 9.29084867e-02 2.78940029e-03
  4.61323895e-02 6.87694475e-02 2.18538776e-01 8.96899477e-02
  6.00793920e-02 7.82104954e-02 1.72191828e-01 5.25694676e-02]
 [3.45482156e-02 3.87243740e-02 1.69577319e-02 1.26550242e-03
  1.07061505e-01 6.96026310e-02 3.23588967e-01 8.47886596e-03
  1.01240189e-03 6.32751212e-02 3.20931405e-01 1.45532778e-02]
 [1.21330721e-02 4.50097844e-02 1.95694715e-02 1.17416831e-03
  8.29745606e-02 1.76125243e-02 2.50489235e-01 4.01174158e-01
  2.34833662e-03 5.67514673e-02 1.01369865e-01 9.39334650e-03]
 [2.09708735e-02 6.60194159e-02 8.05825219e-02 1.14563107e-02
  6.33009672e-02 1.94174761e-04 6.96893215e-01 1.14563107e-02
  1.68932043e-02 2.17475723e-02 5.24271838e-03 5.24271838e-03]
 [8.83826911e-02 4.19134386e-02 2.23234631e-02 1.41230067e-02
  7

In [ ]:
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags)) # 用表來看 左邊為先前的狀態 上面為現在的狀態
display(tags_df)

,X,.,ADP,PRT,ADJ,PRON,NOUN,VERB,CONJ,NUM,DET,ADV
X,0.075726,0.160869,0.142226,0.185086,0.017682,0.054200,0.061695,0.206419,0.010379,0.003075,0.056890,0.025754
.,0.025641,0.092372,0.092908,0.002789,0.046132,0.068769,0.218539,0.089690,0.060079,0.078210,0.172192,0.052569
ADP,0.034548,0.038724,0.016958,0.001266,0.107062,0.069603,0.323589,0.008479,0.001012,0.063275,0.320931,0.014553
PRT,0.012133,0.045010,0.019569,0.001174,0.082975,0.017613,0.250489,0.401174,0.002348,0.056751,0.101370,0.009393
ADJ,0.020971,0.066019,0.080583,0.011456,0.063301,0.000194,0.696893,0.011456,0.016893,0.021748,0.005243,0.005243
PRON,0.088383,0.041913,0.022323,0.014123,0.070615,0.006834,0.212756,0.484738,0.005011,0.006834,0.009567,0.036902
NOUN,0.028825,0.240094,0.176827,0.043935,0.012584,0.004659,0.262344,0.149134,0.042454,0.009144,0.013106,0.016895
VERB,0.215930,0.034807,0.092357,0.030663,0.066390,0.035543,0.110589,0.167956,0.005433,0.022836,0.133610,0.083886
CONJ,0.009330,0.035126,0.055982,0.004391,0.113611,0.060373,0.349067,0.150384,0.000549,0.040615,0.123491,0.057080
NUM,0.202428,0.119243,0.037487,0.026062,0.035345,0.001428,0.351660,0.020707,0.014281,0.184220,0.003570,0.003570


In [ ]:
# list(set([pair[1] for pair in train_tagged_words]))

In [ ]:
def Viterbi(words, train_bag = train_tagged_words):
  state = []
  T = list(set([pair[1] for pair in train_bag]))
    
  for key, word in enumerate(words):
      
    p = [] 
    for tag in T:
      if key == 0:                 # transition_p = 特定狀態發生的機率 (例如:形容詞連接名詞)
        transition_p = tags_df.loc['.', tag]   # 只有一個字的時候 當作前面是 '.' 的狀況
      else:
        transition_p = tags_df.loc[state[-1], tag]  
            
      # 計算 HMM 看狀態的機率跟觀察的機率
      emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
      state_probability = emission_p * transition_p    
      p.append(state_probability)
          
    pmax = max(p)
    state_max = T[p.index(pmax)] 
    state.append(state_max)
  return list(zip(words, state))

In [ ]:
random.seed(1234)      
rndom = [random.randint(1,len(test_set)) for x in range(10)]
test_run = [test_set[i] for i in rndom]
test_run_base = [tup for sent in test_run for tup in sent]
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [ ]:
len(rndom)

10

In [ ]:
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  61.668063163757324
Viterbi Algorithm Accuracy:  94.25837320574163


全部跑 實在太久就先不執行了

In [ ]:
# test_tagged_words = [tup for sent in test_set for tup in sent]
# test_untagged_words = [tup[0] for sent in test_set for tup in sent]
# test_untagged_words
 
# start = time.time()
# tagged_seq = Viterbi(test_untagged_words)
# end = time.time()
# difference = end-start
 
# print("Time taken in seconds: ", difference)
 
# # accuracy
# check = [i for i, j in zip(test_tagged_words, test_untagged_words) if i == j] 
 
# accuracy = len(check)/len(tagged_seq)
# print('Viterbi Algorithm Accuracy: ',accuracy*100)

In [ ]:
patterns = [
    (r'.*ing$', 'VERB'),              # 進行式
    (r'.*ed$', 'VERB'),               # 過去式 
    (r'.*es$', 'VERB'),               # 動詞  
    (r'.*\'s$', 'NOUN'),              # 所有格
    (r'.*s$', 'NOUN'),               # 複數
    (r'\*T?\*?-[0-9]+$', 'X'),           # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'),        # 基數
    (r'.*', 'NOUN')                 # nouns
]
 
rule_based_tagger = nltk.RegexpTagger(patterns)

In [ ]:
def Viterbi_rule_based(words, train_bag = train_tagged_words):
  state = []
  T = list(set([pair[1] for pair in train_bag]))
    
  for key, word in enumerate(words): 
    p = [] 
    for tag in T:
      if key == 0:
        transition_p = tags_df.loc['.', tag]
      else:
        transition_p = tags_df.loc[state[-1], tag]
             
      emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
      state_probability = emission_p * transition_p    
      p.append(state_probability)
          
    pmax = max(p)
    state_max = rule_based_tagger.tag([word])[0][1]       
    
      
    if(pmax==0):
      state_max = rule_based_tagger.tag([word])[0][1] 
    else:
      if state_max != 'X':
        state_max = T[p.index(pmax)]                
          
      
    state.append(state_max)
  return list(zip(words, state))

In [ ]:
start = time.time()
tagged_seq = Viterbi_rule_based(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  67.74631190299988
Viterbi Algorithm Accuracy:  97.1291866028708


比較兩種方法

In [ ]:
test_sent="Will can see Marry"
pred_tags_rule=Viterbi_rule_based(test_sent.split())
pred_tags_withoutRules= Viterbi(test_sent.split())
print(pred_tags_rule)
print(pred_tags_withoutRules)

[('Will', 'NOUN'), ('can', 'VERB'), ('see', 'VERB'), ('Marry', 'NOUN')]
[('Will', 'X'), ('can', 'VERB'), ('see', 'VERB'), ('Marry', 'X')]
